In [ ]:
import time

In [ ]:
import re

In [ ]:
import string

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
import contractions

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
from nltk.corpus import stopwords

In [ ]:
from nltk.corpus import wordnet

In [ ]:
from nltk import pos_tag

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
from nltk.stem import PorterStemmer

In [ ]:
from num2words import num2words

In [ ]:
from spellchecker import SpellChecker

In [ ]:
tweets = pd.read_csv('final.csv').iloc[:,1:]

In [ ]:
tweet = tweets['Tweets']

In [ ]:
tweet[17]

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
stop_words.remove('not')

In [ ]:
punct_words = list(string.punctuation)

In [ ]:
stop = stop_words + punct_words

In [ ]:
stop = list(set(stop))

In [ ]:
def get_easy_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
spell = SpellChecker()

In [ ]:
digitise = lambda x: re.sub('-|\s+', '', num2words(x.group()))

In [ ]:
def f1(text):
    unurl = re.sub('http[s]?://\S+', '', str(text).strip().lower())
    for i in unurl.split():
        if i[0] in ['#', '@']:
            unurl = unurl.replace(i, ' ')
    unspaced = ' '.join(unurl.split()).strip()
    expanded = contractions.fix(unspaced)
    digit_to_word = re.sub(r'\d+', digitise, expanded)
    clean_words = []
    for word in word_tokenize(digit_to_word):
        if word not in stop:
            pos = pos_tag([word])
            clean_word = lemmatizer.lemmatize(word, get_easy_pos(pos[0][1]))
            clean_words.append(clean_word)
    clean_words = ' '.join(clean_words)
    correct_word = spell.correction(clean_words)
    return correct_word

In [ ]:
tweet0 = tweet[:10]

In [ ]:
st = time.time()
new_tweet0 = tweet0.apply(f1)
print('time_taken : ', time.time() - st)

In [ ]:
len(tweet0), len(new_tweet0)

In [ ]:
st = time.time()
new_tweet = tweet.apply(f1)
print('time_taken : ', time.time() - st)

In [ ]:
print('time for preprocessing in min: ', 3032 / 60)

In [ ]:
tweets['Tweets'] = new_tweet

In [ ]:
tweets.head()

In [ ]:
tweets.iloc[78, 0]